In [1]:
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os
import re
import warnings

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

In [2]:
articles = pd.read_csv('articles.csv')
customers = pd.read_csv('customers.csv')
transactions_train = pd.read_csv('transactions_train.csv')
transactions_train['customer_id'] = transactions_train['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
customers['customer_id'] = customers['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
# handle missing values
articles.fillna(value=0, inplace=True)
customers.fillna(value=0, inplace=True)
transactions_train.fillna(value=0, inplace=True)

In [3]:
transactions_train = transactions_train.drop_duplicates(subset=['customer_id','article_id']).reset_index(drop = True)
customer_list = transactions_train.customer_id.value_counts()[:50].index.tolist()
customers_1 = customers.loc[customers['customer_id'].isin(customer_list)]
transactions_train_t = transactions_train.loc[transactions_train['customer_id'].isin(customer_list)]
transactions_train_test = transactions_train_t.loc[transactions_train_t['t_dat']>'2020-04-01'] 
transactions_train_train = transactions_train_t.loc[transactions_train_t['t_dat']<='2020-04-01'] 
transactions_train_test

,t_dat,customer_id,article_id,price,sales_channel_id
20587642,2020-04-02,1135991499650384534,857570001,0.033881,2
20587643,2020-04-02,1135991499650384534,869423001,0.084729,2
20588614,2020-04-02,-738104704126742956,730683041,0.033881,2
20589542,2020-04-02,-3864861336650713034,786304006,0.025407,2
20598202,2020-04-02,-4557035136527070563,857812004,0.042034,2
...,...,...,...,...,...
27304077,2020-09-22,2768447994533782618,889661001,0.066186,1
27304078,2020-09-22,2768447994533782618,865799005,0.033085,1
27304079,2020-09-22,2768447994533782618,918525001,0.049627,1
27304974,2020-09-22,2636587246054202986,752814020,0.033881,2


In [4]:
articles_t = articles.loc[articles['article_id'].isin(list(transactions_train_t['article_id']))]

In [5]:
merge_df = pd.DataFrame()
for customer in customers_1['customer_id']:
    new_df = pd.DataFrame()
    new_df['Buy'] = []
    #print(customer)
    x = [customers_1.loc[customers_1['customer_id']==customer]]*len(articles_t)
    #print(type(x))
    #new_df = new_df[x].reset_index(drop = True)
    new_df = pd.concat(x).reset_index(drop = True)#(pd.DataFrame(x)).reset_index(drop = True)
    print(new_df)

    #new_df = new_df.loc([customers_1.loc[customers_1['customer_id']==customer]]*len(articles_t)).reset_index(drop = True)

#     new_df = new_df.append([customers_1.loc[customers_1['customer_id']==customer]]*len(articles_t)).reset_index(drop = True)
    new_df['article_id'] = list(articles_t['article_id'])
    tran_temp = transactions_train_t.loc[transactions_train_t['customer_id']==customer]
    for article in list(tran_temp['article_id']):
        new_df.loc[(new_df['customer_id']==customer) & (new_df['article_id']==article),['Buy']] = 1

    #merge_df = pd.Series(new_df).reset_index(drop=True)
    merge_df = pd.concat([merge_df,new_df]).reset_index(drop = True)


               customer_id   FN  Active club_member_status   
0      1135991499650384534  1.0     1.0             ACTIVE  \
1      1135991499650384534  1.0     1.0             ACTIVE   
2      1135991499650384534  1.0     1.0             ACTIVE   
3      1135991499650384534  1.0     1.0             ACTIVE   
4      1135991499650384534  1.0     1.0             ACTIVE   
...                    ...  ...     ...                ...   
22919  1135991499650384534  1.0     1.0             ACTIVE   
22920  1135991499650384534  1.0     1.0             ACTIVE   
22921  1135991499650384534  1.0     1.0             ACTIVE   
22922  1135991499650384534  1.0     1.0             ACTIVE   
22923  1135991499650384534  1.0     1.0             ACTIVE   

      fashion_news_frequency   age   
0                  Regularly  51.0  \
1                  Regularly  51.0   
2                  Regularly  51.0   
3                  Regularly  51.0   
4                  Regularly  51.0   
...                      ..

In [6]:
merge_df['Buy'] = merge_df['Buy'].fillna(0)
merge_df= pd.merge(merge_df, articles, on='article_id', how='left')
print(merge_df)

                 customer_id   FN  Active club_member_status   
0        1135991499650384534  1.0     1.0             ACTIVE  \
1        1135991499650384534  1.0     1.0             ACTIVE   
2        1135991499650384534  1.0     1.0             ACTIVE   
3        1135991499650384534  1.0     1.0             ACTIVE   
4        1135991499650384534  1.0     1.0             ACTIVE   
...                      ...  ...     ...                ...   
1146195  2636587246054202986  1.0     1.0             ACTIVE   
1146196  2636587246054202986  1.0     1.0             ACTIVE   
1146197  2636587246054202986  1.0     1.0             ACTIVE   
1146198  2636587246054202986  1.0     1.0             ACTIVE   
1146199  2636587246054202986  1.0     1.0             ACTIVE   

        fashion_news_frequency   age   
0                    Regularly  51.0  \
1                    Regularly  51.0   
2                    Regularly  51.0   
3                    Regularly  51.0   
4                    Regularly 

In [7]:
x = merge_df.drop(columns=['Buy','FN','Active','club_member_status','fashion_news_frequency', 'postal_code', 'prod_name','product_type_name','product_group_name','graphical_appearance_name','colour_group_name','perceived_colour_value_name','perceived_colour_master_name','department_name','index_name','index_group_name','section_name','garment_group_name','detail_desc','index_code']) 
y = merge_df['Buy']


# x = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
# y = np.array([1, 1, 2, 2])
#x = x.values.reshape(-1, 1)
#x = x.transpose()
#y = y.values.reshape(1,-1)
#y = y.transpose()

# print(x.shape)
# print(y.shape)

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.8, random_state=42)
sgd = SGDClassifier(alpha=1, max_iter=10000)
sgd.fit(x_train,y_train)

SGDClassifier(alpha=1, max_iter=10000)

In [8]:
prediction = sgd.predict(x_test)

In [9]:
from sklearn.metrics import accuracy_score
# Reference: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html


acc = accuracy_score(y_test, prediction)
print('Accuracy:', acc)

Accuracy: 0.499075205025301


In [86]:
# top 12 recs
new_df = pd.DataFrame()
if '5854009424779598107' in x_test['customer_id'].values: #=='5854009424779598107':
    new_df['customer_id'] = x_test['customer_id']
    new_df['article_id'] = x_test['article_id']
    new_df['product_code'] = x_test['product_code']
    new_df['model_output'] = prediction
print(new_df.head(12))



                 customer_id  article_id  product_code  model_output
957265   5854009424779598107   819113007        819113           1.0
1056829  5854009424779598107   580102004        580102           0.0
728664   5854009424779598107   828067002        828067           1.0
746543   5854009424779598107   759108001        759108           0.0
208701   5854009424779598107   581781010        581781           0.0
331606   5854009424779598107   733098018        733098           0.0
790956   5854009424779598107   742561002        742561           0.0
429650   5854009424779598107   814586001        814586           1.0
1054842  5854009424779598107   398829008        398829           0.0
425265   5854009424779598107   755458006        755458           1.0
8987     5854009424779598107   711701001        711701           0.0
362053   5854009424779598107   831070001        831070           1.0
